# Clock and Timer の開発
- まずはレイアウトを作成

In [5]:
import PySimpleGUI as sg
from pyautogui import size
import platform  # add chatGPT
import datetime as dt
import time


FOCUS_MINUTES, FOCUS_SECONDS = (0, 3)
BREAK_MINUTES, BREAK_SECONDS = (0, 2)

today = dt.datetime.today()
focus_time = dt.datetime.combine(today, dt.time(0, FOCUS_MINUTES, FOCUS_SECONDS))
focus_time_str = focus_time.strftime("%M:%S")
break_time = dt.datetime.combine(today, dt.time(0, BREAK_MINUTES, BREAK_SECONDS))

# OSごとの時間・日付フォーマットの設定
if platform.system() == "Windows":
    time_format = "%H:%M"
    sec_format = "%S"
    date_format = "%#Y/%#m/%#d"
elif platform.system() == "Linux" or platform.system() == "Darwin":
    time_format = "%H:%M"
    sec_format = "%S"
    date_format = "%Y/%m/%d"


def time_formats():
    now = dt.datetime.now()
    hours_minutes = now.strftime(time_format)
    sec = now.strftime(sec_format)
    date = now.strftime(date_format)
    weekday = now.strftime("%a")
    timer_update = True
    return hours_minutes, sec, date, weekday, timer_update

# タイマーの初期化
def get_time(minutes, seconds):
    today = dt.datetime.today()
    return dt.datetime.combine(today, dt.time(0, minutes, seconds))

# タイマーのリセット機能
def reset_timer():
    focus_time = get_time(FOCUS_MINUTES, FOCUS_SECONDS)
    break_time = get_time(BREAK_MINUTES, BREAK_SECONDS)
    return focus_time, break_time


hours_minutes, sec, date, weekday, timer_update = time_formats()
focus_time, break_time = reset_timer()

# time, sec, date, weekday, timer_update = "09:00", "00", "2024/12/30", "Sat", False
AUTO_CLOSE_DURATION = 15
display_text = 'R'


timer_layout = [
    sg.Button(
        f'{focus_time_str} {display_text}',
        key="-TIMER_BTN-",
        font=("impact", 20),
    ),
    sg.Text(f'R', font=('impact', 20), key='-MSG-'),
    sg.Button(
        "Reset",
        key="-RESET_BTN-",
        font=("impact", 20),
        # pad=((1, 0), (15, 0)),
    ),
]

clock_layout = (
    [
        sg.Text(
            hours_minutes,
            key="-TIME-",
            font=("impact", 50),
            # text_color="gray",
            # pad=((0, 0), (0, 0)),
            size=(5, 1),
            # auto_size_text=False,
            justification='center',
        ),
        # sg.Text(
        #     sec,
        #     key="-SEC-",
        #     font=("impact", 25),
        #     # text_color="gray",
        #     pad=((0, 0), (35, 0)),
        #     size=(3, 1),
        #     justification='left',
        #     # auto_size_text=False,
        # ),
    ],
)

date_layout = (
    [
        sg.Text(
            date,
            key="-DATE-",
            font=("impact", 20),
            # text_color="gray",
        ),
        sg.Text(
            weekday,
            key="-WEEKDAY-",
            font=("impact", 20),
            # text_color="gray",
        ),
    ],
)


layout = [
    timer_layout,
    clock_layout,
    date_layout,
]

window = sg.Window(
    title="Clock and Pomodoro Timer",
    layout=layout,
    size=(400, 300),
    location=(200, 200),
    # location=(size()[0] - 370, size()[1] - 380),
    element_justification="center",
    # transparent_color=sg.theme_background_color(),
    # no_titlebar=True,
    right_click_menu=["menu", ["Exit", "!Properties"]],
    keep_on_top=True,
    grab_anywhere_using_control=True,
    # alpha_channel=0.5,
    auto_close=True,
    auto_close_duration=AUTO_CLOSE_DURATION,
)

timer_start = False
focus_start = True

while True:
    event, values = window.read(timeout=1000, timeout_key="-TIMEOUT-")
    
    # break
    if event in [sg.WIN_CLOSED, "Exit"]:
        break

    # clock
    if event in "-TIMEOUT-" and timer_update:
        hours_minutes, sec, date, weekday, timer_update = time_formats()
        window["-TIME-"].update(hours_minutes)
        # window["-SEC-"].update(sec)
        window["-DATE-"].update(f"{date}")
        window["-WEEKDAY-"].update(f"{weekday}")
    
    # timer
    if event == 'Reset':
        timer_start = False
        focus_start = True
        focus_time, break_time = reset_timer()
        window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")}')  # ボタンの表示も初期化
        # window['-MSG-'].update('READY')  # メッセージも初期化
        display_text = 'R'

    if event == '-TIMER_BTN-':
        timer_start = not timer_start  # ボタンでスタート・ストップの切り替え

    if timer_start:
        if focus_start:
            focus_time_str = focus_time.strftime("%M:%S")
            if focus_time_str == "59:59":
                focus_start = False  # 作業時間が終了したら休憩に切り替え
                focus_time = get_time(FOCUS_MINUTES, FOCUS_SECONDS)  # 作業時間をリセット
            else:
                window['-TIMER_BTN-'].update(f'{focus_time_str} {display_text}')
                # window['-MSG-'].update('F')
                display_text = 'F'
                focus_time -= dt.timedelta(seconds=1)
        else:
            break_time_str = break_time.strftime("%M:%S")
            if break_time_str == "59:59":
                timer_start = False  # 休憩時間が終了したらタイマーを停止して初期化
                focus_start = True  # 初期化して作業に切り替え
                focus_time, break_time = reset_timer()  # タイマーを初期状態にリセット
                display_text = 'R'
                window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")} {display_text}')  # ボタンの表示も初期化
                # window['-MSG-'].update('R')  # メッセージも初期化
            else:
                display_text = 'B'
                window['-TIMER_BTN-'].update(f'{break_time_str} {display_text}')
                # window['-MSG-'].update('B')
                break_time -= dt.timedelta(seconds=1)
    

window.close()

In [8]:
import PySimpleGUI as sg

col = [
    [sg.Text("Row1 col1")],
    [sg.Text("Row2 col1")],
    [sg.Text("Row3 col1")],
    [sg.Text("Row4 col1")],
]

layout = [[sg.Column(col)]]

while True:
    event, values = sg.Window("sg column test", layout).Read()
    if event == sg.WIN_CLOSED:
        break

In [9]:
import PySimpleGUI as sg

col = [
    [sg.Text('Row1 col1'), sg.Text('Row1 col2'), sg.Text('Row1 col3')],
]

layout = [[sg.Column(col)]]

while True:
    event, values = sg.Window('sg column test', layout).Read()
    if event == sg.WIN_CLOSED:
        break

In [43]:
import PySimpleGUI as sg

col = [
    [sg.Text('Row1 col1'), sg.Text('Row1 col2'), sg.Text('Row1 col3')],
    [sg.Text('Row2 col1'), sg.Text('Row2 col2'), sg.Text('Row2 col3')],
    [sg.Text('Row3 col1'), sg.Text('Row3 col2'), sg.Text('Row3 col3')],
]

layout = [[sg.Column(col)]]

while True:
    event, values = sg.Window('sg column test', layout).Read()
    if event == sg.WIN_CLOSED:
        break

In [86]:
import PySimpleGUI as sg

col1 = [
    [sg.Text('col1 Row 1', font=('', 30)), sg.Text('col1 Row 2')],
]

# col2 = [
#     [sg.Text('col2 Row 1'), sg.Text('col2-2 row 1')],
#     [sg.Text('col2 Row 2')],
#     [sg.Text('col2 Row 3'), sg.Text('col2-2 row 3'), sg.Text('col2-3 row 3')]
# ]

col3 = [
    [sg.Text('col3 Row 1')],
    [sg.Text('col3 Row 2')]
]

layout = [
    [sg.Column(col1, vertical_alignment='bottom')],
    [sg.Column(col3)],
    # [sg.Text('No Column Text'), sg.Text('No Column Text 2')],
    [sg.OK()]
]

event, values = sg.Window('sg column test', layout).Read()

In [1]:
#!usr/bin/env python
# -*- coding: utf-8 -*-

import PySimpleGUI as sg

sg.ChangeLookAndFeel("BlueMono")

# 　列要素レイアウト（1段目：テキスト、2段目：テキスト、テキスト入力欄、3段目：テキスト、テキスト入力欄）
col = [
    [sg.Button("列２")],
    [sg.Text("列３"),],
]

# 　レイアウト（1段目：リストボックス、列要素レイアウト、2段目：テキスト入力欄、3段目：ボタン）
layout = [
    [sg.Text("aaaa", size=(10, 3)), sg.Column(col, background_color="blue")],
    # [sg.OK()],
]

# 　ウィンドウの生成と、入力待ち（二つの処理を一つにまとめて書いています）
event, values = sg.Window("複数列を表示するサンプルアプリ", layout).Read()